In [20]:
from preprocess import process, convert_to_open


df = convert_to_open()

In [19]:
%load_ext autoreload
%autoreload 2

In [2]:
df

,convertible,question,answers
0,True,What are the impacts of a warming climate on t...,[Global warming causes melting of ice on land ...
1,True,What is ice and snow albedo?,[Albedo is the measure of how much light that ...
2,True,How does the cryosphere interact with the atmo...,[The atmosphere and the cryosphere are two key...
3,True,How are earth observations of the cryosphere u...,[To monitor shipping routes and reduce the ris...
4,True,What is the benefit of monitoring the cryosphe...,[SAR instruments are not weather dependent and...
...,...,...,...
428,True,Excluding observations from satellite and in s...,"[Seasonal predictions, climate reanalysis and ..."
429,True,What is ERA5 data?,[Historical observations into global estimates...
430,True,What are climate projections?,[Simulations on the climate until the end of t...
431,True,What are the C3S Climate Data Store Applications?,[Pre-built interactive visualisations of the d...


In [5]:
# Select the rows with more than one answer
df_multiple = df[df['answers'].apply(lambda x: len(x) > 1)]
# Drop convertible column
df_multiple = df_multiple.drop(columns=['convertible'])
# Save to csv
df_multiple.to_csv('multiple_answers.csv', index=False)

In [21]:
df_single = df[df['answers'].apply(lambda x: len(x) == 1)]
# Drop convertible column
df_single = df_single.drop(columns=['convertible'])
# Rename answers to answer
df_single = df_single.rename(columns={'answers': 'answer'})
# Convert the answer to string
df_single['answer'] = df_single['answer'].apply(lambda x: x[0])
df_single

,question,answer
1,What is ice and snow albedo?,Albedo is the measure of how much light that h...
2,How does the cryosphere interact with the atmo...,The atmosphere and the cryosphere are two key ...
4,What is the benefit of monitoring the cryosphe...,SAR instruments are not weather dependent and ...
5,What is the difference between sea ice and ice...,"Sea ice forms on ocean water, but icebergs bre..."
6,How does melting sea ice affect the ocean’s ci...,"When sea ice melts, it makes the sea water les..."
...,...,...
308,Excluding observations from satellite and in s...,"Seasonal predictions, climate reanalysis and c..."
309,What is ERA5 data?,Historical observations into global estimates ...
310,What are climate projections?,Simulations on the climate until the end of th...
311,What are the C3S Climate Data Store Applications?,Pre-built interactive visualisations of the data


In [22]:
import pandas as pd
# Read the multiple answers csv
df_multiple = pd.read_csv('openqa.csv')
df_multiple

,question,answer
0,What are the impacts of a warming climate on t...,Global warming causes melting of ice on land s...
1,How are earth observations of the cryosphere u...,Earth observation are used to: monitor shippin...
2,What is special about the Copernicus programme...,With Copernicus programme multiple satellites ...
3,Why are satellites important for studying clim...,Satellites provide global coverage of measurem...
4,Why do we need long-term data records?,We need long-term data records to: detect tren...
...,...,...
76,Why can cloud classification be challenging to...,Satellite Passive instruments cannot measure c...
77,Why is Sentinel-1’s Radar instrument useful fo...,It can measure through clouds to show us the c...
78,What could changing regional climates alter?,"Forests, crop yields, water supply."
79,What are the three key services provided by C3S?,"The CDS, Data in action, Climate Bulletins"


In [26]:
# Find missing indices in data1
full_index = range(min(df_single.index)-1, max(df_single.index) + 1)  # Full range of indices
missing_indices = sorted(set(full_index) - set(df_single.index))  # Indices that are missing
len(missing_indices)

81

In [27]:
df_multiple.index = missing_indices
df_multiple

,question,answer
0,What are the impacts of a warming climate on t...,Global warming causes melting of ice on land s...
3,How are earth observations of the cryosphere u...,Earth observation are used to: monitor shippin...
23,What is special about the Copernicus programme...,With Copernicus programme multiple satellites ...
24,Why are satellites important for studying clim...,Satellites provide global coverage of measurem...
25,Why do we need long-term data records?,We need long-term data records to: detect tren...
...,...,...
294,Why can cloud classification be challenging to...,Satellite Passive instruments cannot measure c...
296,Why is Sentinel-1’s Radar instrument useful fo...,It can measure through clouds to show us the c...
305,What could changing regional climates alter?,"Forests, crop yields, water supply."
306,What are the three key services provided by C3S?,"The CDS, Data in action, Climate Bulletins"


In [28]:

# Merge datasets
merged = pd.concat([df_single, df_multiple]).sort_index()

In [30]:
# Save to csv
merged.to_csv('is_open_q&a.csv', index=False)

In [32]:
correct_answers = [['A', 'B', 'D'], ['C']]
predictions = [['A'], ['C', 'B']]
from sklearn.metrics import precision_score, recall_score, f1_score

def precision_recall_f1_set_based(correct_answers, predictions):
    flat_correct = []
    flat_pred = []
    
    for correct, pred in zip(correct_answers, predictions):
        # Convert answers to sets for each question
        flat_correct.extend(correct)
        flat_pred.extend(pred)
    
    precision = precision_score(flat_correct, flat_pred, average='micro')
    recall = recall_score(flat_correct, flat_pred, average='micro')
    f1 = f1_score(flat_correct, flat_pred, average='micro')
    
    return precision, recall, f1

precision, recall, f1 = precision_recall_f1_set_based(correct_answers, predictions)
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-Score: {f1}")


ValueError: Found input variables with inconsistent numbers of samples: [4, 3]

In [33]:
def subset_accuracy(references, predictions):
    correct_count = 0
    for correct, pred in zip(references, predictions):
        if set(correct) == set(pred):
            correct_count += 1
    return correct_count / len(references)

correct_answers = [['A', 'B', 'D'], ['C']]
predictions = [['A'], ['C', 'B']]
subset_accuracy(correct_answers, predictions)

0.0

In [36]:
set([[1, 2, 3]]) == set([[3, 2, 1]])

TypeError: unhashable type: 'list'

In [37]:
import re
def get_answers(answer):
    answers_list = []
    matches = re.findall(r"(?:The answer is: |The answers are: )([A-Z](?:,\s*[A-Z])*)", answer)
    if matches:
        match = matches[0]
        # Split on spaces and strip
        answers_list = [ans.strip() for ans in match.split(",")]
    return answers_list

get_answers("The answer is: A, B, D")

['A', 'B', 'D']

## Loading custom format to HF

In [2]:
%load_ext autoreload
%autoreload 2

In [15]:
import datasets
from preprocess import process

data = process()
data

Uploading the dataset shards: 100%|██████████| 1/1 [00:00<00:00,  1.12it/s]


Dataset({
    features: ['question', 'choices', 'answers'],
    num_rows: 432
})

In [16]:
data.push_to_hub('eve-esa/eve-is-mcqa', private=True)
# Push csv file

Uploading the dataset shards: 100%|██████████| 1/1 [00:00<00:00,  1.30it/s]


CommitInfo(commit_url='https://huggingface.co/datasets/eve-esa/eve-is-mcqa/commit/8aeef9c9b376a67f5a933a624a40a412354acfdf', commit_message='Upload dataset', commit_description='', oid='8aeef9c9b376a67f5a933a624a40a412354acfdf', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/eve-esa/eve-is-mcqa', endpoint='https://huggingface.co', repo_type='dataset', repo_id='eve-esa/eve-is-mcqa'), pr_revision=None, pr_num=None)

In [18]:
datasets.load_dataset('eve-esa/eve-is-mcqa')['train']


Dataset({
    features: ['question', 'choices', 'answers'],
    num_rows: 432
})

In [12]:
from huggingface_hub import HfApi

api = HfApi()
api.upload_file(
    path_or_fileobj="IS_Q&A_w_convertible.csv",
    path_in_repo="IS_Q&A_w_convertible.csv",
    repo_id="eve-esa/eve-is-mcqa",
    repo_type="dataset",
)

CommitInfo(commit_url='https://huggingface.co/datasets/eve-esa/eve-is-mcqa/commit/09d493ab0b088fedad22231738e024687431f043', commit_message='Upload IS_Q&A_w_convertible.csv with huggingface_hub', commit_description='', oid='09d493ab0b088fedad22231738e024687431f043', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/eve-esa/eve-is-mcqa', endpoint='https://huggingface.co', repo_type='dataset', repo_id='eve-esa/eve-is-mcqa'), pr_revision=None, pr_num=None)

## Convert to open-ended


In [27]:
from preprocess import convert_to_open

df = convert_to_open()
df

Dataset({
    features: ['question', 'answer'],
    num_rows: 313
})

In [29]:
# Upload on HF

df.push_to_hub('eve-esa/eve-is-open-ended', private=True)

Uploading the dataset shards: 100%|██████████| 1/1 [00:01<00:00,  1.37s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/eve-esa/eve-is-open-ended/commit/f81377550cd2cb5510670025f3ea9375f3166476', commit_message='Upload dataset', commit_description='', oid='f81377550cd2cb5510670025f3ea9375f3166476', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/eve-esa/eve-is-open-ended', endpoint='https://huggingface.co', repo_type='dataset', repo_id='eve-esa/eve-is-open-ended'), pr_revision=None, pr_num=None)

In [32]:
datasets.load_dataset('eve-esa/eve-is-open-ended')

Generating train split: 100%|██████████| 313/313 [00:00<00:00, 92903.34 examples/s]


DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 313
    })
})